# Default

In [8]:
import os
import numpy as np 
import pandas as pd

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

# scipy
from scipy.stats import boxcox, yeojohnson
from scipy.special import inv_boxcox

# sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV, GroupKFold
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.ensemble import StackingRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import *

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.graphics.mosaicplot import mosaic

import tensorflow as tf
import json
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.callbacks import EarlyStopping, ModelCheckpoint


from statistics import stdev

import xgboost as xgb 
import lightgbm as lgb

import itertools
from korean_lunar_calendar import KoreanLunarCalendar

# 폰트 설정 - 윈도우
plt.rcParams['font.family'] = 'NanumGothic'

# 그래프의 크기
plt.rcParams['figure.figsize'] = 12, 6

# 글자 크기
plt.rcParams['font.size'] = 14

# 폰트 설정 시 - 기호 깨는거 방지하기
plt.rcParams['axes.unicode_minus'] = False

import seaborn as sns

# 불필요한 경고 메시지를 나오지 않도록 한다.
import warnings
warnings.filterwarnings('ignore')

# 절기 구분을 위한 모듈
import datetime
dateformat = '%Y-%m-%d'

In [9]:
BASE_DIR = './data'

In [10]:
train_path = os.path.join(BASE_DIR, '2021 빅콘테스트_데이터분석분야_퓨처스리그_홍수ZERO_댐유입량,강우,수위데이터_210902_update.xlsx')

data = pd.read_excel(train_path)
data = data[1 : ]
data = data.reset_index(drop = True)
data.iloc[:,6:] = data.iloc[:,6:].apply(pd.to_numeric)  # 수치형으로 변환

data.columns = ['홍수사상번호', '연', '월', '일', '시간', '유입량', 
                '1_유역평균강수', '1_강우(A지역)', '1_강우(B지역)', '1_강우(C지역)', '1_강우(D지역)', '1_수위(E지역)', '1_수위(D지역)', 
                '2_유역평균강수', '2_강우(A지역)', '2_강우(B지역)', '2_강우(C지역)', '2_강우(D지역)', '2_수위(E지역)', '2_수위(D지역)', 
                '3_유역평균강수', '3_강우(A지역)', '3_강우(B지역)', '3_강우(C지역)', '3_강우(D지역)', '3_수위(E지역)', '3_수위(D지역)',
                '4_유역평균강수', '4_강우(A지역)', '4_강우(B지역)', '4_강우(C지역)', '4_강우(D지역)', '4_수위(E지역)', '4_수위(D지역)',
                '5_유역평균강수', '5_강우(A지역)', '5_강우(B지역)', '5_강우(C지역)', '5_강우(D지역)', '5_수위(E지역)', '5_수위(D지역)',
                '6_유역평균강수', '6_강우(A지역)', '6_강우(B지역)', '6_강우(C지역)', '6_강우(D지역)', '6_수위(E지역)', '6_수위(D지역)']

## 기온

In [11]:
weather_path = os.path.join(BASE_DIR, '기상데이터.csv')

weather_data = pd.read_csv(weather_path, encoding = 'euc-kr')
print('기상데이터 적용 전 :', data.shape)
data = pd.merge(data, weather_data,left_index=True, right_index=True, how='left')

print('기상데이터 적용 후 :', data.shape)

기상데이터 적용 전 : (3051, 48)
기상데이터 적용 후 : (3051, 52)


In [14]:
data_feat = data

In [15]:
data_7 = data_feat[['1_유역평균강수', '2_유역평균강수', '3_유역평균강수', '4_유역평균강수', '5_유역평균강수', '6_유역평균강수']]
lst = []
lst_mean = []
for i in range(len(data_7)) :
  lst.append(list(data_7.loc[i]))
  lst[i].sort()
  lst[i].pop()
  lst[i].sort(reverse = True)
  lst[i].pop()
  lst_mean.append(np.mean(lst[i]))

data_feat_7 = pd.DataFrame(lst_mean, columns = ['7_유역평균강수'])

data_7 = data_feat[['1_강우(A지역)', '2_강우(A지역)', '3_강우(A지역)', '4_강우(A지역)', '5_강우(A지역)', '6_강우(A지역)']]
lst = []
lst_mean = []
for i in range(len(data_7)) :
  lst.append(list(data_7.loc[i]))
  lst[i].sort()
  lst[i].pop()
  lst[i].sort(reverse = True)
  lst[i].pop()
  lst_mean.append(np.mean(lst[i]))

data_feat_7 = pd.concat([data_feat_7, pd.DataFrame(lst_mean, columns = ['7_강우(A지역)'])], axis = 1)

data_7 = data_feat[['1_강우(B지역)', '2_강우(B지역)', '3_강우(B지역)', '4_강우(B지역)', '5_강우(B지역)', '6_강우(B지역)']]
lst = []
lst_mean = []
for i in range(len(data_7)) :
  lst.append(list(data_7.loc[i]))
  lst[i].sort()
  lst[i].pop()
  lst[i].sort(reverse = True)
  lst[i].pop()
  lst_mean.append(np.mean(lst[i]))

data_feat_7 = pd.concat([data_feat_7, pd.DataFrame(lst_mean, columns = ['7_강우(B지역)'])], axis = 1)

data_7 = data_feat[['1_강우(C지역)', '2_강우(C지역)', '3_강우(C지역)', '4_강우(C지역)', '5_강우(C지역)', '6_강우(C지역)']]
lst = []
lst_mean = []
for i in range(len(data_7)) :
  lst.append(list(data_7.loc[i]))
  lst[i].sort()
  lst[i].pop()
  lst[i].sort(reverse = True)
  lst[i].pop()
  lst_mean.append(np.mean(lst[i]))

data_feat_7 = pd.concat([data_feat_7, pd.DataFrame(lst_mean, columns = ['7_강우(C지역)'])], axis = 1)

data_7 = data_feat[['1_강우(D지역)', '2_강우(D지역)', '3_강우(D지역)', '4_강우(D지역)', '5_강우(D지역)', '6_강우(D지역)']]
lst = []
lst_mean = []
for i in range(len(data_7)) :
  lst.append(list(data_7.loc[i]))
  lst[i].sort()
  lst[i].pop()
  lst[i].sort(reverse = True)
  lst[i].pop()
  lst_mean.append(np.mean(lst[i]))

data_feat_7 = pd.concat([data_feat_7, pd.DataFrame(lst_mean, columns = ['7_강우(D지역)'])], axis = 1)

data_7 = data_feat[['1_수위(D지역)', '2_수위(D지역)', '3_수위(D지역)', '4_수위(D지역)', '5_수위(D지역)', '6_수위(D지역)']]
lst = []
lst_mean = []
for i in range(len(data_7)) :
  lst.append(list(data_7.loc[i]))
  lst[i].sort()
  lst[i].pop()
  lst[i].sort(reverse = True)
  lst[i].pop()
  lst_mean.append(np.mean(lst[i]))

data_feat_7 = pd.concat([data_feat_7, pd.DataFrame(lst_mean, columns = ['7_수위(D지역)'])], axis = 1)
data_feat = pd.concat([data_feat, data_feat_7], axis = 1)

## rolling

In [16]:
# rolling 1
data_rolling = data_feat[['홍수사상번호']]
lst = [data_rolling[data_rolling['홍수사상번호'] == i].iloc[0 : 1].index for i in range(1, 27)]
lst = list(itertools.chain.from_iterable(lst))

data_rolling['7_유역평균강수_shift_1'] = data_feat['7_유역평균강수'].shift(1)
data_rolling['7_강우(A지역)_shift_1'] = data_feat['7_강우(A지역)'].shift(1)
data_rolling['7_강우(B지역)_shift_1'] = data_feat['7_강우(B지역)'].shift(1)
data_rolling['7_강우(C지역)_shift_1'] = data_feat['7_강우(C지역)'].shift(1)
data_rolling['7_강우(D지역)_shift_1'] = data_feat['7_강우(D지역)'].shift(1)
data_rolling['7_수위(D지역)_shift_1'] = data_feat['7_수위(D지역)'].shift(1)
data_rolling['7_수위(E지역)_shift_1'] = data_feat['1_수위(E지역)'].shift(1)
for i in range(1, 27) :
    dx = data_rolling[data_rolling['홍수사상번호'] == i]['7_유역평균강수_shift_1'].iloc[1] - (data_rolling[data_rolling['홍수사상번호'] == i]['7_유역평균강수_shift_1'].iloc[2] - data_rolling[data_rolling['홍수사상번호'] == i]['7_유역평균강수_shift_1'].iloc[1])
    if dx < 0 :
        data_rolling.loc[lst[i-1], '7_유역평균강수_shift_1'] = 0
    else :
        data_rolling.loc[lst[i-1], '7_유역평균강수_shift_1'] = dx

    dx = data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(A지역)_shift_1'].iloc[1] - (data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(A지역)_shift_1'].iloc[2] - data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(A지역)_shift_1'].iloc[1])
    if dx < 0 :
        data_rolling.loc[lst[i-1], '7_강우(A지역)_shift_1'] = 0
    else :
        data_rolling.loc[lst[i-1], '7_강우(A지역)_shift_1'] = dx

    dx = data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(B지역)_shift_1'].iloc[1] - (data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(B지역)_shift_1'].iloc[2] - data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(B지역)_shift_1'].iloc[1])
    if dx < 0 :
        data_rolling.loc[lst[i-1], '7_강우(B지역)_shift_1'] = 0
    else :
        data_rolling.loc[lst[i-1], '7_강우(B지역)_shift_1'] = dx

    dx = data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(C지역)_shift_1'].iloc[1] - (data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(C지역)_shift_1'].iloc[2] - data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(C지역)_shift_1'].iloc[1])
    if dx < 0 :
        data_rolling.loc[lst[i-1], '7_강우(C지역)_shift_1'] = 0
    else :
        data_rolling.loc[lst[i-1], '7_강우(C지역)_shift_1'] = dx

    dx = data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(D지역)_shift_1'].iloc[1] - (data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(D지역)_shift_1'].iloc[2] - data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(D지역)_shift_1'].iloc[1])
    if dx < 0 :
        data_rolling.loc[lst[i-1], '7_강우(D지역)_shift_1'] = 0
    else :
        data_rolling.loc[lst[i-1], '7_강우(D지역)_shift_1'] = dx

    dx = data_rolling[data_rolling['홍수사상번호'] == i]['7_수위(D지역)_shift_1'].iloc[1] - (data_rolling[data_rolling['홍수사상번호'] == i]['7_수위(D지역)_shift_1'].iloc[2] - data_rolling[data_rolling['홍수사상번호'] == i]['7_수위(D지역)_shift_1'].iloc[1])
    if dx < 0 :
        data_rolling.loc[lst[i-1], '7_수위(D지역)_shift_1'] = 0
    else :
        data_rolling.loc[lst[i-1], '7_수위(D지역)_shift_1'] = dx

    dx = data_rolling[data_rolling['홍수사상번호'] == i]['7_수위(E지역)_shift_1'].iloc[1] - (data_rolling[data_rolling['홍수사상번호'] == i]['7_수위(E지역)_shift_1'].iloc[2] - data_rolling[data_rolling['홍수사상번호'] == i]['7_수위(E지역)_shift_1'].iloc[1])
    if dx < 0 :
        data_rolling.loc[lst[i-1], '7_수위(E지역)_shift_1'] = 0
    else :
        data_rolling.loc[lst[i-1], '7_수위(E지역)_shift_1'] = dx

data_rolling = data_rolling.drop(['홍수사상번호'], axis = 1)

data_feat = pd.concat([data_feat, data_rolling], axis = 1)

In [17]:
# rolling 2
data_rolling = data_feat[['홍수사상번호']]
lst = [data_rolling[data_rolling['홍수사상번호'] == i].iloc[0 : 1].index for i in range(1, 27)]
lst = list(itertools.chain.from_iterable(lst))

data_rolling['7_유역평균강수_shift_2'] = data_feat['7_유역평균강수_shift_1'].shift(1)
data_rolling['7_강우(A지역)_shift_2'] = data_feat['7_강우(A지역)_shift_1'].shift(1)
data_rolling['7_강우(B지역)_shift_2'] = data_feat['7_강우(B지역)_shift_1'].shift(1)
data_rolling['7_강우(C지역)_shift_2'] = data_feat['7_강우(C지역)_shift_1'].shift(1)
data_rolling['7_강우(D지역)_shift_2'] = data_feat['7_강우(D지역)_shift_1'].shift(1)
data_rolling['7_수위(D지역)_shift_2'] = data_feat['7_수위(D지역)_shift_1'].shift(1)
data_rolling['7_수위(E지역)_shift_2'] = data_feat['7_수위(E지역)_shift_1'].shift(1)
for i in range(1, 27) :
    dx = data_rolling[data_rolling['홍수사상번호'] == i]['7_유역평균강수_shift_2'].iloc[1] - (data_rolling[data_rolling['홍수사상번호'] == i]['7_유역평균강수_shift_2'].iloc[2] - data_rolling[data_rolling['홍수사상번호'] == i]['7_유역평균강수_shift_2'].iloc[1])
    if dx < 0 :
        data_rolling.loc[lst[i-1], '7_유역평균강수_shift_2'] = 0
    else :
        data_rolling.loc[lst[i-1], '7_유역평균강수_shift_2'] = dx
        
    dx = data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(A지역)_shift_2'].iloc[1] - (data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(A지역)_shift_2'].iloc[2] - data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(A지역)_shift_2'].iloc[1])
    if dx < 0 :
        data_rolling.loc[lst[i-1], '7_강우(A지역)_shift_2'] = 0
    else :
        data_rolling.loc[lst[i-1], '7_강우(A지역)_shift_2'] = dx

    dx = data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(B지역)_shift_2'].iloc[1] - (data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(B지역)_shift_2'].iloc[2] - data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(B지역)_shift_2'].iloc[1])
    if dx < 0 :
        data_rolling.loc[lst[i-1], '7_강우(B지역)_shift_2'] = 0
    else :
        data_rolling.loc[lst[i-1], '7_강우(B지역)_shift_2'] = dx

    dx = data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(C지역)_shift_2'].iloc[1] - (data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(C지역)_shift_2'].iloc[2] - data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(C지역)_shift_2'].iloc[1])
    if dx < 0 :
        data_rolling.loc[lst[i-1], '7_강우(C지역)_shift_2'] = 0
    else :
        data_rolling.loc[lst[i-1], '7_강우(C지역)_shift_2'] = dx

    dx = data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(D지역)_shift_2'].iloc[1] - (data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(D지역)_shift_2'].iloc[2] - data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(D지역)_shift_2'].iloc[1])
    if dx < 0 :
        data_rolling.loc[lst[i-1], '7_강우(D지역)_shift_2'] = 0
    else :
        data_rolling.loc[lst[i-1], '7_강우(D지역)_shift_2'] = dx

    dx = data_rolling[data_rolling['홍수사상번호'] == i]['7_수위(D지역)_shift_2'].iloc[1] - (data_rolling[data_rolling['홍수사상번호'] == i]['7_수위(D지역)_shift_2'].iloc[2] - data_rolling[data_rolling['홍수사상번호'] == i]['7_수위(D지역)_shift_2'].iloc[1])
    if dx < 0 :
        data_rolling.loc[lst[i-1], '7_수위(D지역)_shift_2'] = 0
    else :
        data_rolling.loc[lst[i-1], '7_수위(D지역)_shift_2'] = dx

    dx = data_rolling[data_rolling['홍수사상번호'] == i]['7_수위(E지역)_shift_2'].iloc[1] - (data_rolling[data_rolling['홍수사상번호'] == i]['7_수위(E지역)_shift_2'].iloc[2] - data_rolling[data_rolling['홍수사상번호'] == i]['7_수위(E지역)_shift_2'].iloc[1])
    if dx < 0 :
        data_rolling.loc[lst[i-1], '7_수위(E지역)_shift_2'] = 0
    else :
        data_rolling.loc[lst[i-1], '7_수위(E지역)_shift_2'] = dx

data_rolling = data_rolling.drop(['홍수사상번호'], axis = 1)
data_feat = pd.concat([data_feat, data_rolling], axis = 1)

In [18]:
# rolling 3
data_rolling = data_feat[['홍수사상번호']]
lst = [data_rolling[data_rolling['홍수사상번호'] == i].iloc[0 : 1].index for i in range(1, 27)]
lst = list(itertools.chain.from_iterable(lst))

data_rolling['7_유역평균강수_shift_3'] = data_feat['7_유역평균강수_shift_2'].shift(1)
data_rolling['7_강우(A지역)_shift_3'] = data_feat['7_강우(A지역)_shift_2'].shift(1)
data_rolling['7_강우(B지역)_shift_3'] = data_feat['7_강우(B지역)_shift_2'].shift(1)
data_rolling['7_강우(C지역)_shift_3'] = data_feat['7_강우(C지역)_shift_2'].shift(1)
data_rolling['7_강우(D지역)_shift_3'] = data_feat['7_강우(D지역)_shift_2'].shift(1)
data_rolling['7_수위(D지역)_shift_3'] = data_feat['7_수위(D지역)_shift_2'].shift(1)
data_rolling['7_수위(E지역)_shift_3'] = data_feat['7_수위(E지역)_shift_2'].shift(1)
for i in range(1, 27) :
    dx = data_rolling[data_rolling['홍수사상번호'] == i]['7_유역평균강수_shift_3'].iloc[1] - (data_rolling[data_rolling['홍수사상번호'] == i]['7_유역평균강수_shift_3'].iloc[2] - data_rolling[data_rolling['홍수사상번호'] == i]['7_유역평균강수_shift_3'].iloc[1])
    if dx < 0 :
        data_rolling.loc[lst[i-1], '7_유역평균강수_shift_3'] = 0
    else :
        data_rolling.loc[lst[i-1], '7_유역평균강수_shift_3'] = dx

    dx = data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(A지역)_shift_3'].iloc[1] - (data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(A지역)_shift_3'].iloc[2] - data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(A지역)_shift_3'].iloc[1])
    if dx < 0 :
        data_rolling.loc[lst[i-1], '7_강우(A지역)_shift_3'] = 0
    else :
        data_rolling.loc[lst[i-1], '7_강우(A지역)_shift_3'] = dx

    dx = data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(B지역)_shift_3'].iloc[1] - (data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(B지역)_shift_3'].iloc[2] - data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(B지역)_shift_3'].iloc[1])
    if dx < 0 :
        data_rolling.loc[lst[i-1], '7_강우(B지역)_shift_3'] = 0
    else :
        data_rolling.loc[lst[i-1], '7_강우(B지역)_shift_3'] = dx

    dx = data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(C지역)_shift_3'].iloc[1] - (data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(C지역)_shift_3'].iloc[2] - data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(C지역)_shift_3'].iloc[1])
    if dx < 0 :
        data_rolling.loc[lst[i-1], '7_강우(C지역)_shift_3'] = 0
    else :
        data_rolling.loc[lst[i-1], '7_강우(C지역)_shift_3'] = dx

    dx = data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(D지역)_shift_3'].iloc[1] - (data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(D지역)_shift_3'].iloc[2] - data_rolling[data_rolling['홍수사상번호'] == i]['7_강우(D지역)_shift_3'].iloc[1])
    if dx < 0 :
        data_rolling.loc[lst[i-1], '7_강우(D지역)_shift_3'] = 0
    else :
        data_rolling.loc[lst[i-1], '7_강우(D지역)_shift_3'] = dx

    dx = data_rolling[data_rolling['홍수사상번호'] == i]['7_수위(D지역)_shift_3'].iloc[1] - (data_rolling[data_rolling['홍수사상번호'] == i]['7_수위(D지역)_shift_3'].iloc[2] - data_rolling[data_rolling['홍수사상번호'] == i]['7_수위(D지역)_shift_3'].iloc[1])
    if dx < 0 :
        data_rolling.loc[lst[i-1], '7_수위(D지역)_shift_3'] = 0
    else :
        data_rolling.loc[lst[i-1], '7_수위(D지역)_shift_3'] = dx

    dx = data_rolling[data_rolling['홍수사상번호'] == i]['7_수위(E지역)_shift_3'].iloc[1] - (data_rolling[data_rolling['홍수사상번호'] == i]['7_수위(E지역)_shift_3'].iloc[2] - data_rolling[data_rolling['홍수사상번호'] == i]['7_수위(E지역)_shift_3'].iloc[1])
    if dx < 0 :
        data_rolling.loc[lst[i-1], '7_수위(E지역)_shift_3'] = 0
    else :
        data_rolling.loc[lst[i-1], '7_수위(E지역)_shift_3'] = dx

data_rolling = data_rolling.drop(['홍수사상번호'], axis = 1)
data_feat = pd.concat([data_feat, data_rolling], axis = 1)

## Difference

In [19]:
data_feat['7_유역평균강수_diff'] = pd.DataFrame(data_feat['7_유역평균강수'] - data_feat['7_유역평균강수_shift_1'])
data_feat['7_강우(A지역)_diff'] = pd.DataFrame(data_feat['7_강우(A지역)'] - data_feat['7_강우(A지역)_shift_1'])
data_feat['7_강우(B지역)_diff'] = pd.DataFrame(data_feat['7_강우(B지역)'] - data_feat['7_강우(B지역)_shift_1'])
data_feat['7_강우(C지역)_diff'] = pd.DataFrame(data_feat['7_강우(C지역)'] - data_feat['7_강우(C지역)_shift_1'])
data_feat['7_강우(D지역)_diff'] = pd.DataFrame(data_feat['7_강우(D지역)'] - data_feat['7_강우(D지역)_shift_1'])
data_feat['7_수위(D지역)_diff'] = pd.DataFrame(data_feat['7_수위(D지역)'] - data_feat['7_수위(D지역)_shift_1'])
data_feat['7_수위(E지역)_diff'] = pd.DataFrame(data_feat['1_수위(E지역)'] - data_feat['7_수위(E지역)_shift_1'])

In [20]:
data_feat

,홍수사상번호,연,월,일,시간,유입량,1_유역평균강수,1_강우(A지역),1_강우(B지역),1_강우(C지역),...,7_강우(D지역)_shift_3,7_수위(D지역)_shift_3,1_수위(E지역)_shift_3,7_유역평균강수_diff,7_강우(A지역)_diff,7_강우(B지역)_diff,7_강우(C지역)_diff,7_강우(D지역)_diff,7_수위(D지역)_diff,1_수위(E지역)_diff
0,1.0,2006.0,7.0,10.0,8.0,189.100000,6.4000,7,7,7,...,5.0,122.612125,2.57,0.475000,0.0,1.0,1.00,1.00,-0.004979,-0.01
1,1.0,2006.0,7.0,10.0,9.0,216.951962,6.3000,7,8,7,...,6.0,122.607146,2.56,0.475000,0.0,1.0,1.00,1.00,-0.004979,-0.01
2,1.0,2006.0,7.0,10.0,10.0,251.424419,6.4000,7,9,7,...,7.0,122.602167,2.55,0.900000,0.0,1.0,0.25,0.50,-0.004458,0.00
3,1.0,2006.0,7.0,10.0,11.0,302.812199,7.3000,7,10,7,...,8.0,122.597188,2.54,2.000000,1.0,1.0,3.25,2.00,-0.001083,0.00
4,1.0,2006.0,7.0,10.0,12.0,384.783406,8.2000,7,12,8,...,9.0,122.592208,2.53,3.100000,2.5,2.0,2.00,1.75,-0.004417,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3046,26.0,2018.0,7.0,7.0,17.0,NaN,2.3689,1,0,0,...,0.0,129.925417,3.21,0.000000,0.0,0.0,-0.25,0.00,0.011875,-0.02
3047,26.0,2018.0,7.0,7.0,18.0,NaN,2.3689,1,0,0,...,0.0,129.939375,3.19,0.000000,0.0,0.0,0.00,0.00,0.013208,-0.01
3048,26.0,2018.0,7.0,7.0,19.0,NaN,2.3689,1,0,0,...,0.0,129.957229,3.18,0.000000,0.0,0.0,0.00,0.00,0.007062,-0.02
3049,26.0,2018.0,7.0,7.0,20.0,NaN,2.3689,1,0,0,...,0.0,129.969104,3.16,-0.005025,0.0,0.0,0.00,0.00,0.007063,-0.02


## 홍수기간 칼럼 생성

In [21]:
rain = data_feat.groupby('홍수사상번호')['시간'].count()
lst = np.array(rain.tolist())

flood = []
for i in range(26):
    x = list(range(1, lst[i]+1))
    flood.extend(x)

flood = pd.DataFrame(flood)
flood.columns = ['홍수기간']
data_feat = pd.concat([data_feat, flood], axis = 1)

data_feat

,홍수사상번호,연,월,일,시간,유입량,1_유역평균강수,1_강우(A지역),1_강우(B지역),1_강우(C지역),...,7_수위(D지역)_shift_3,1_수위(E지역)_shift_3,7_유역평균강수_diff,7_강우(A지역)_diff,7_강우(B지역)_diff,7_강우(C지역)_diff,7_강우(D지역)_diff,7_수위(D지역)_diff,1_수위(E지역)_diff,홍수기간
0,1.0,2006.0,7.0,10.0,8.0,189.100000,6.4000,7,7,7,...,122.612125,2.57,0.475000,0.0,1.0,1.00,1.00,-0.004979,-0.01,1
1,1.0,2006.0,7.0,10.0,9.0,216.951962,6.3000,7,8,7,...,122.607146,2.56,0.475000,0.0,1.0,1.00,1.00,-0.004979,-0.01,2
2,1.0,2006.0,7.0,10.0,10.0,251.424419,6.4000,7,9,7,...,122.602167,2.55,0.900000,0.0,1.0,0.25,0.50,-0.004458,0.00,3
3,1.0,2006.0,7.0,10.0,11.0,302.812199,7.3000,7,10,7,...,122.597188,2.54,2.000000,1.0,1.0,3.25,2.00,-0.001083,0.00,4
4,1.0,2006.0,7.0,10.0,12.0,384.783406,8.2000,7,12,8,...,122.592208,2.53,3.100000,2.5,2.0,2.00,1.75,-0.004417,0.00,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3046,26.0,2018.0,7.0,7.0,17.0,NaN,2.3689,1,0,0,...,129.925417,3.21,0.000000,0.0,0.0,-0.25,0.00,0.011875,-0.02,156
3047,26.0,2018.0,7.0,7.0,18.0,NaN,2.3689,1,0,0,...,129.939375,3.19,0.000000,0.0,0.0,0.00,0.00,0.013208,-0.01,157
3048,26.0,2018.0,7.0,7.0,19.0,NaN,2.3689,1,0,0,...,129.957229,3.18,0.000000,0.0,0.0,0.00,0.00,0.007062,-0.02,158
3049,26.0,2018.0,7.0,7.0,20.0,NaN,2.3689,1,0,0,...,129.969104,3.16,-0.005025,0.0,0.0,0.00,0.00,0.007063,-0.02,159


## 홍수기간 Binning

In [22]:
bins = [0,30, 50, 70, 100, 120, 160, 220] 
data_feat['홍수_bin'] = np.digitize(data_feat['홍수기간'], bins)
data_feat.groupby('홍수_bin')[['홍수기간']].count()

,홍수기간
홍수_bin,
1,754
2,494
3,416
4,480
5,196
6,263
7,287
8,161


In [23]:
data_feat

,홍수사상번호,연,월,일,시간,유입량,1_유역평균강수,1_강우(A지역),1_강우(B지역),1_강우(C지역),...,1_수위(E지역)_shift_3,7_유역평균강수_diff,7_강우(A지역)_diff,7_강우(B지역)_diff,7_강우(C지역)_diff,7_강우(D지역)_diff,7_수위(D지역)_diff,1_수위(E지역)_diff,홍수기간,홍수_bin
0,1.0,2006.0,7.0,10.0,8.0,189.100000,6.4000,7,7,7,...,2.57,0.475000,0.0,1.0,1.00,1.00,-0.004979,-0.01,1,1
1,1.0,2006.0,7.0,10.0,9.0,216.951962,6.3000,7,8,7,...,2.56,0.475000,0.0,1.0,1.00,1.00,-0.004979,-0.01,2,1
2,1.0,2006.0,7.0,10.0,10.0,251.424419,6.4000,7,9,7,...,2.55,0.900000,0.0,1.0,0.25,0.50,-0.004458,0.00,3,1
3,1.0,2006.0,7.0,10.0,11.0,302.812199,7.3000,7,10,7,...,2.54,2.000000,1.0,1.0,3.25,2.00,-0.001083,0.00,4,1
4,1.0,2006.0,7.0,10.0,12.0,384.783406,8.2000,7,12,8,...,2.53,3.100000,2.5,2.0,2.00,1.75,-0.004417,0.00,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3046,26.0,2018.0,7.0,7.0,17.0,NaN,2.3689,1,0,0,...,3.21,0.000000,0.0,0.0,-0.25,0.00,0.011875,-0.02,156,6
3047,26.0,2018.0,7.0,7.0,18.0,NaN,2.3689,1,0,0,...,3.19,0.000000,0.0,0.0,0.00,0.00,0.013208,-0.01,157,6
3048,26.0,2018.0,7.0,7.0,19.0,NaN,2.3689,1,0,0,...,3.18,0.000000,0.0,0.0,0.00,0.00,0.007062,-0.02,158,6
3049,26.0,2018.0,7.0,7.0,20.0,NaN,2.3689,1,0,0,...,3.16,-0.005025,0.0,0.0,0.00,0.00,0.007063,-0.02,159,6


## KMEANS

In [24]:
# kmeans를 하기위해서는 변수들이 수치형데이터여야 한다. 그리고 타겟변수인 유입량도 빼준다.
# 그리고 시간의 영향을 제거해보기 위해 시간관련 열들도 제거해본다
data_feat_target = data_feat[data_feat['홍수사상번호'] == 26]
data_feat = data_feat[data_feat['홍수사상번호'] != 26]
data_feat_kmeans = data_feat.drop(['홍수사상번호', '연', '월', '일', '시간', '일시', '음력', '음력_월', '음력_일', '유입량'], axis = 1)
data_feat_target_kmeans = data_feat_target.drop(['홍수사상번호', '연', '월', '일', '시간', '일시', '음력', '음력_월', '음력_일', '유입량'], axis = 1)

In [25]:
# create model and prediction
scaler = StandardScaler()
model = KMeans(n_clusters=2,algorithm='auto')
pipeline = make_pipeline(scaler,model)
pipeline.fit(data_feat_kmeans)

predict_target = pd.DataFrame(pipeline.predict(data_feat_target_kmeans))
predict_target.columns=['kmeans']
predict = pd.DataFrame(pipeline.predict(data_feat_kmeans))
predict.columns=['kmeans']

data_feat = pd.concat([pd.concat([data_feat, predict], axis = 1), pd.concat([data_feat_target.reset_index(), predict_target], axis = 1).set_index('index')])

In [26]:
data_feat

,홍수사상번호,연,월,일,시간,유입량,1_유역평균강수,1_강우(A지역),1_강우(B지역),1_강우(C지역),...,7_유역평균강수_diff,7_강우(A지역)_diff,7_강우(B지역)_diff,7_강우(C지역)_diff,7_강우(D지역)_diff,7_수위(D지역)_diff,1_수위(E지역)_diff,홍수기간,홍수_bin,kmeans
0,1.0,2006.0,7.0,10.0,8.0,189.100000,6.4000,7,7,7,...,0.475000,0.0,1.0,1.00,1.00,-0.004979,-0.01,1,1,0
1,1.0,2006.0,7.0,10.0,9.0,216.951962,6.3000,7,8,7,...,0.475000,0.0,1.0,1.00,1.00,-0.004979,-0.01,2,1,0
2,1.0,2006.0,7.0,10.0,10.0,251.424419,6.4000,7,9,7,...,0.900000,0.0,1.0,0.25,0.50,-0.004458,0.00,3,1,0
3,1.0,2006.0,7.0,10.0,11.0,302.812199,7.3000,7,10,7,...,2.000000,1.0,1.0,3.25,2.00,-0.001083,0.00,4,1,0
4,1.0,2006.0,7.0,10.0,12.0,384.783406,8.2000,7,12,8,...,3.100000,2.5,2.0,2.00,1.75,-0.004417,0.00,5,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3046,26.0,2018.0,7.0,7.0,17.0,NaN,2.3689,1,0,0,...,0.000000,0.0,0.0,-0.25,0.00,0.011875,-0.02,156,6,0
3047,26.0,2018.0,7.0,7.0,18.0,NaN,2.3689,1,0,0,...,0.000000,0.0,0.0,0.00,0.00,0.013208,-0.01,157,6,0
3048,26.0,2018.0,7.0,7.0,19.0,NaN,2.3689,1,0,0,...,0.000000,0.0,0.0,0.00,0.00,0.007062,-0.02,158,6,0
3049,26.0,2018.0,7.0,7.0,20.0,NaN,2.3689,1,0,0,...,-0.005025,0.0,0.0,0.00,0.00,0.007063,-0.02,159,6,0


## 절기

In [27]:
data_time = data_feat[['연', '월', '일', '시간']]

data_time['연'] = [str(int(i)) for i in data_time['연']]
data_time['월'] = [str(int(i)) for i in data_time['월']]
data_time['일'] = [str(int(i)) for i in data_time['일']]
data_time['시간'] = [str(int(i)) for i in data_time['시간']]

In [28]:
# 7월 8일 기준으로 일 차이를 열로 만든다 (소서)
data_time['월일'] = data_time['월'] + '-' + data_time['일']
data_time['월일'] = [datetime.datetime.strptime(i, '%m-%d')  for i in data_time['월일']]
data_time['월일'] = data_time['월일'] - datetime.datetime.strptime('7-8', '%m-%d')
data_time['월일'] = [i.days for i in data_time['월일']]
data_time['절기'] = data_time['월일']
data_time['절기'] = [i // 15 for i in data_time['절기']]

In [29]:
# 6/28 부터 9/24일 까지 시작일 포함 총 89일.
for i in range(len(data_time)) :
    if data_time['절기'].iloc[i] < 0 :
        data_time.loc[i, '절기'] = '하지'
    elif data_time['절기'].iloc[i] == 0 :
        data_time.loc[i, '절기'] = '소서'
    elif data_time['절기'].iloc[i] == 1 :
        data_time.loc[i, '절기'] = '대서'
    elif data_time['절기'].iloc[i] == 2 :
        data_time.loc[i, '절기'] = '입추'
    elif data_time['절기'].iloc[i] == 3 :
        data_time.loc[i, '절기'] = '처서'
    elif data_time['절기'].iloc[i] == 4 :
        data_time.loc[i, '절기'] = '백로'
    else :
        data_time.loc[i, '절기'] = '추분'

In [30]:
data_time = data_time[['절기']]
data_feat = pd.concat([data_feat, data_time], axis = 1)

# LSTM

In [37]:
py_data = data_feat.copy()
py_data['홍수사상번호'] = py_data['홍수사상번호'].astype('object')
py_data = py_data.drop(['일시', '음력', '음력_월', '음력_일', '절기', '연', '월', '일', '시간'], axis = 1)

target = py_data[py_data['홍수사상번호'] == 26].reset_index(drop = True)
y_target = py_data[py_data['홍수사상번호'] == 26][['유입량']].reset_index(drop = True)
train = py_data[py_data['홍수사상번호'] != 26]
y_train = py_data[py_data['홍수사상번호'] != 26][['유입량']]

# 스케일링
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = MinMaxScaler()

cat_columns = [c for c, t in zip(train.dtypes.index, train.dtypes) if t=='O'] 
scale_cols = [c for c in train.drop(['유입량'],axis = 1).columns if c not in cat_columns]

# train 정규화 수행
scaled_df = scaler.fit_transform(train[scale_cols])
scaled_df = pd.DataFrame(scaled_df, columns=scale_cols)
train = train.drop(scaled_df.columns, axis = 1)

scaled_train = pd.concat([train, scaled_df, y_train], axis = 1)


cat_columns = [c for c, t in zip(target.dtypes.index, target.dtypes) if t=='O'] 
scale_cols = [c for c in target.drop(['유입량'],axis = 1).columns if c not in cat_columns]

# target 정규화 수행
scaled_df = scaler.transform(target[scale_cols])
scaled_df = pd.DataFrame(scaled_df, columns=scale_cols)
target = target.drop(scaled_df.columns, axis = 1)

scaled_target = pd.concat([target, scaled_df, y_target], axis = 1)

# sequence dataset 만드는 함수 설정
def make_sequene_dataset(feature, label, window_size):
    
    feature_list = []
    label_list = []
    
    for i in range(len(feature)-window_size):
        # feature[i:i+window_size] 슬라이싱을 이용하여 
        # [[...], [...], ..] 형상으로 입력데이터, 즉 feature 생성
        feature_list.append(feature[i:i+window_size])
        label_list.append(label[i+window_size])
        
    return np.array(feature_list), np.array(label_list)

def MAPE(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100 

def rmse(y_pred, y_true):
    RMSE = mean_squared_error(y_true, y_pred)**0.5
    return RMSE

def train_val_test(X, X_target, target, n) :    
    X['홍수사상번호'] = X['홍수사상번호'].astype('int')
    
    # Train, Valid, Test set 설정
    X_target_y = X_target[[target]]
    X_target = X_target.drop([target], axis = 1)

    X_y = X[[target]]
    X = X.drop([target], axis = 1)
    
    box_result = boxcox(X_y[target])
    X_y[target] = boxcox(X_y[target])[0]

    train_pred = X.copy()
    train_pred = train_pred[['홍수사상번호']]

    result = []
    result_rmse = []
    group_kfold = GroupKFold(n_splits = n)
    groups = X['홍수사상번호'].values
    group_kfold.get_n_splits(X, X_y, groups = groups)
    i = 1

    X = X.drop(['홍수사상번호'], axis = 1)
    X_target = X_target.drop(['홍수사상번호'], axis = 1)

    for train_index, test_index in group_kfold.split(X, X_y, groups):
        print("{}번째 교차검증 {} /".format(i, i), n)
        X_train, X_val = X.iloc[train_index], X.iloc[test_index]
        y_train, y_val = X_y.iloc[train_index], X_y.iloc[test_index] 

        # 딥러닝 학습을 위해 numpy로 변환
        label_np = y_train.to_numpy()
        feature_np = X_train.to_numpy()

        label_valid_np = y_val.to_numpy()
        feature_valid_np = X_val.to_numpy()

        label_test_np = X_target_y.to_numpy()
        feature_test_np = X_target.to_numpy()

        window_size = 5
        x_train, y_train = make_sequene_dataset(feature_np, label_np, window_size)
        x_valid, y_valid = make_sequene_dataset(feature_valid_np, label_valid_np, window_size)
        x_test, y_test = make_sequene_dataset(feature_test_np, label_test_np, window_size)
        
        # LSTM  모델 구축
        model = Sequential()

        model.add(LSTM(526, activation = 'tanh', 
                       input_shape = x_train[0].shape))
        model.add(Dense(256))               
        model.add(Dense(1, activation='linear'))
        model.summary()

        model.compile(loss='mse', optimizer='adam', metrics=['mae'])
        early_stop = EarlyStopping(monitor = 'val_loss', patience = 5)
        model.fit(x_train, y_train, validation_data = (x_valid, y_valid), epochs = 1000, verbose=0,
                  batch_size = 200, callbacks = [early_stop])


        pred = inv_boxcox(model.predict(x_valid), box_result[1])
        train_pred.loc[test_index[5:],'lstm'] = pred

        i += 1
        
    return train_pred

result = train_val_test(train, target, '유입량', n = 24) 

1번째 교차검증 1 / 24
Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_24 (LSTM)               (None, 526)               1108808   
_________________________________________________________________
dense_48 (Dense)             (None, 256)               134912    
_________________________________________________________________
dense_49 (Dense)             (None, 1)                 257       
Total params: 1,243,977
Trainable params: 1,243,977
Non-trainable params: 0
_________________________________________________________________
2번째 교차검증 2 / 24
Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_25 (LSTM)               (None, 526)               1108808   
_________________________________________________________________
dense_50 (Dense)             (None, 256)              

12번째 교차검증 12 / 24
Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_35 (LSTM)               (None, 526)               1108808   
_________________________________________________________________
dense_70 (Dense)             (None, 256)               134912    
_________________________________________________________________
dense_71 (Dense)             (None, 1)                 257       
Total params: 1,243,977
Trainable params: 1,243,977
Non-trainable params: 0
_________________________________________________________________
13번째 교차검증 13 / 24
Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_36 (LSTM)               (None, 526)               1108808   
_________________________________________________________________
dense_72 (Dense)             (None, 256)          

23번째 교차검증 23 / 24
Model: "sequential_46"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_46 (LSTM)               (None, 526)               1108808   
_________________________________________________________________
dense_92 (Dense)             (None, 256)               134912    
_________________________________________________________________
dense_93 (Dense)             (None, 1)                 257       
Total params: 1,243,977
Trainable params: 1,243,977
Non-trainable params: 0
_________________________________________________________________
24번째 교차검증 24 / 24
Model: "sequential_47"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_47 (LSTM)               (None, 526)               1108808   
_________________________________________________________________
dense_94 (Dense)             (None, 256)          

In [38]:
result

,홍수사상번호,lstm
0,1,NaN
1,1,NaN
2,1,NaN
3,1,NaN
4,1,NaN
...,...,...
2886,25,1044.252563
2887,25,1044.252563
2888,25,1044.252563
2889,25,1044.252075


window size를 5로 설정하였기 때문에 사상 별 첫 5개의 관측치가 결측값으로 나오게 됩니다. 이것을 해결하기위해, 머신러닝으로 앞서 5개의 관측치를 예측하여 채우는 방식을 고려했습니다.